In [1]:
import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import networkx as nx 
import nupack as nu
from random import choice as choicerandom
from numpy.random import choice as choicenumpy
from hdna import * 

In [3]:
M = Model('dna', '3D', min_nucleation=3, sliding_cutoff=2, stacking='nostacking')
# A = Strand(M, 'GGGGGGGGGGGGGGGGG')
A = Strand(M, 'CAACAACACCACCA')
# A = Strand(M, 'ATGATGTTCCGCTTT')
B = A.complementary()

In [4]:
G = Geometry(120,270)
C = Chamber(M, A, B)
N = Kinetwork(M, A, B)
K = Kinetics(M, N, G)
S = Simulator(M, N, K)

In [18]:
N = Kinetwork(M, A, B, clean=True)

In [23]:
N.add_nodes()
list(N.Graph.nodes.data())

[(<hdna.complex.Complex at 0x7f98a47aa650>,
  {'object': <hdna.complex.Complex at 0x7f98a47aa650>,
   'structure': '..............+..............',
   'state': 'singlestranded',
   'pairs': 0}),
 (<hdna.complex.Sliding at 0x7f98a479d5a0>,
  {'object': <hdna.complex.Sliding at 0x7f98a479d5a0>,
   'structure': '(((((.(((((...+))))).)))))...',
   'state': 'backfray',
   'pairs': 10,
   'dpxdist': 3}),
 (<hdna.complex.Zippo at 0x7f98a479dc00>,
  {'object': <hdna.complex.Zippo at 0x7f98a479dc00>,
   'structure': '....(.(.......+....).).......',
   'state': 'backfray',
   'pairs': 2}),
 (<hdna.complex.Zippo at 0x7f98a4792fe0>,
  {'object': <hdna.complex.Zippo at 0x7f98a4792fe0>,
   'structure': '...((.((......+...)).))......',
   'state': 'backfray',
   'pairs': 4}),
 (<hdna.complex.Zippo at 0x7f98a4793e20>,
  {'object': <hdna.complex.Zippo at 0x7f98a4793e20>,
   'structure': '..(((.(((.....+..))).))).....',
   'state': 'backfray',
   'pairs': 6}),
 (<hdna.complex.Zippo at 0x7f98a47919c0>,
 

In [5]:
for c in C.offcores:
    print(c.structure, c)
    for b in c.backfray:
        print(b.structure, b)
    print('\n')

(((((.(((((...+))))).)))))... <hdna.complex.Sliding object at 0x7f98c72642e0>
....(.(.......+....).)....... <hdna.complex.Zippo object at 0x7f98c72645b0>
...((.((......+...)).))...... <hdna.complex.Zippo object at 0x7f98c7266200>
..(((.(((.....+..))).)))..... <hdna.complex.Zippo object at 0x7f98c7276ad0>
.((((.((((....+.)))).)))).... <hdna.complex.Zippo object at 0x7f98c7274400>
(((((.(((((...+))))).)))))... <hdna.complex.Sliding object at 0x7f98c72642e0>


...(((((.(((((+...))))).))))) <hdna.complex.Sliding object at 0x7f98c7275f90>
.......(.(....+.......).).... <hdna.complex.Zippo object at 0x7f98c7276c80>
......((.((...+......)).))... <hdna.complex.Zippo object at 0x7f98c7275480>
.....(((.(((..+.....))).))).. <hdna.complex.Zippo object at 0x7f98c7266050>
....((((.((((.+....)))).)))). <hdna.complex.Zippo object at 0x7f98c7247a90>
...(((((.(((((+...))))).))))) <hdna.complex.Sliding object at 0x7f98c7275f90>




In [6]:
from itertools import pairwise

a = [1,2,3,4,5,6,7,8,9]
for e, k in pairwise(a):
    print(e)

1
2
3
4
5
6
7
8


In [7]:
list(N.Graph.nodes)

['..............+..............',
 '(((((.(((((...+))))).)))))...',
 '....(.(.......+....).).......',
 '...((.((......+...)).))......',
 '..(((.(((.....+..))).))).....',
 '.((((.((((....+.)))).))))....',
 '...(((((.(((((+...))))).)))))',
 '.......(.(....+.......).)....',
 '......((.((...+......)).))...',
 '.....(((.(((..+.....))).)))..',
 '....((((.((((.+....)))).)))).',
 '(((...........+...........)))',
 '.(((..........+..........))).',
 '..(((.........+.........)))..',
 '...(((........+........)))...',
 '....(((.......+.......)))....',
 '.....(((......+......))).....',
 '......(((.....+.....)))......',
 '.......(((....+....))).......',
 '........(((...+...)))........',
 '.........(((..+..))).........',
 '..........(((.+.)))..........',
 '...........(((+)))...........',
 '((((..........+..........))))',
 '(((((.........+.........)))))',
 '((((((........+........))))))',
 '(((((((.......+.......)))))))',
 '((((((((......+......))))))))',
 '(((((((((.....+.....)))))))))',
 '((((((((((..

In [8]:
N.save_graph('.')
S.DiGraph()

<class 'hdna.complex.Complex'>
..............+..............
singlestranded
0
<class 'hdna.complex.Sliding'>


KeyError: '(((((.(((((...+))))).)))))...'

In [ ]:
for slide in C.offcores:
    for e in slide.backfray: 
        print(e.structure, e.G)
    print('\n')



..(...........+...).......... -0.4881872600222883
.(((..........+..)))......... -4.120449068824481
(((((.........+.)))))........ -6.925764238903476




.((...........+......))...... -2.546662191507054
((((..........+.....))))..... -5.681043942272813




......((......+...))......... -2.546662191507054
.....((((.....+..))))........ -5.681043942272813
(....((((.(...+).))))....)... -0.38376681830054116






.........((...+......))...... -2.546662191507054
........((((..+.....))))..... -5.681043942272813
...(....((((.(+...).))))....) -2.450460353686646




......((......+...........)). -2.546662191507054
.....((((.....+..........)))) -5.681043942272813




..........(...+...........).. -0.4881872600222883
.........(((..+..........))). -4.120449068824481
........(((((.+.........))))) -6.925764238903476






In [ ]:
a = [1,2,3,4,5,6,7]
a[:-1]

[1, 2, 3, 4, 5, 6]

In [ ]:
import numpy as np 
slide = C.offcores[5]
maxG = np.argmin(slide.gbackfray)
print(maxG, slide.gbackfray[maxG])
print(*zip(slide.sbackfray, slide.gbackfray), sep='\n')
print(slide.backfray[maxG])


1 -5.681043942272813
('......((......+...)).........', -2.546662191507054)
('.....((((.....+..))))........', -5.681043942272813)
('(....((((.(...+).))))....)...', -0.38376681830054116)


In [ ]:
DG = S.DiGraph()
DF = pd.DataFrame(DG.nodes.data('object'))
DF = DF.set_axis(['structure', 'object'], axis=1)
DF['gibbs'] = [obj.G for obj in DF['object']]
PF = DF[DF['gibbs'] < 0]
for s, data in PF.iterrows():
    if type(data['object']) == Sliding:
        print(s)
        print(data['object'].structure)

14
((............+))............
15
............((+............))
18
..(.(((.......+))).).........
19
.........(.(((+.......))).)..
29
(((((.........+.)))))........
30
((((..........+.....)))).....
32
(....((((.(...+).))))....)...
34
...(....((((.(+...).))))....)
36
.....((((.....+..........))))
37
........(((((.+.........)))))


In [ ]:
def get_ix(string, char):
            indices = []
            for i, e in enumerate(list(string)):
                if e == char:
                    indices.append(i)
            return indices 

In [ ]:
import itertools as it 
a = '..............)))))))............'
ix = get_ix(a, ')')
for d, i in it.zip_longest(ix[:int(len(ix)/2)], ix[int(len(ix)/2):][::-1]):
    try: a = a[:d] + '.' + a[d+1:]
    except TypeError: pass 
    try: a = a[:i] + '.' + a[i+1:]
    except TypeError: pass
    print(a)

...............))))).............
................)))..............
.................)...............
.................................


In [ ]:
slide = PF.loc[25,'object']
slide.backfraying_trajectory()
for e in slide.backfrayings:
    print(e)
print(slide.structure)

AttributeError: 'Zippo' object has no attribute 'backfraying_trajectory'

In [ ]:
nupackmodel = nu.Model(
    material='dna', 
    ensemble='nostacking', 
    celsius=25, 
    sodium=1, 
    magnesium=0) 

nuStrand1 = nu.Strand(A.sequence, name = 'a')
nuStrand2 = nu.Strand(B.sequence, name = 'b') # An inversion here is needed because in this program strands are defined as 5-3 against 3-5 but in NUPACK all strands are defined 5-3 and the program takes care to turn them around and so on
nuStructure = nu.Structure('.......((........+........)).......')
dG = nu.structure_energy(strands=[nuStrand1,nuStrand2], structure=nuStructure, model=nupackmodel)
# dGloop = nupackmodel.loop_energy(loop='AAA+TTT', structure='(((+)))')
dG

RuntimeError: C++: : "pair list length doesn't match sequence length" (State.cc, line 27)
len(pairs) = 34
n = 32
